In [1]:
%matplotlib widget

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyodbc
import os
import time
from matplotlib import colors
from scipy import stats

In [3]:
server = 'mrwhite.intrans.iastate.edu'
database = 'sim_Andalib'
cnxn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes')

In [4]:
font = {'family' : 'serif',
        'serif' : 'Times New Roman',
        'size'   : 8}
plt.rc('font', **font)

In [5]:
### input parameters
number_of_intersection = 8

sim_id = 21 

cycle_length = 120

simulation_start = 300
simulation_time = 3600
number_of_cycle = simulation_time//cycle_length
cycle_remainder = simulation_time%cycle_length


In [6]:
filePath = "C:\\Users\\ashams\\Box\\research works\\comparison signal offset optimization\\bandwidth maximization\\"
input_parameters = pd.read_excel(filePath + "input_parameters.xlsx", sheet_name = "38th_subset")

input_parameters.loc[:, 'nextIntersection'] = input_parameters.LinkToInt
input_parameters.loc[input_parameters.sg_id == 6, 'nextIntersection'] -= 2


# input_parameters.loc[:, "offset_loc"] = 0
# input_parameters.loc[:, "offset_up"] = 0
input_parameters.LinkFromInt -= 1
input_parameters.LinkToInt -= 1

input_parameters.loc[:, "travel time"] = np.round(input_parameters.loc[:, "travel time"] , 0)
input_parameters.loc[:, "travel time"] = input_parameters.loc[:, "travel time"]%cycle_length

input_parameters

,Intersection Number,intersection_id,sg_id,tt_id,LinkFromInt,LinkToInt,travel_time,saturation flow,number of lane,vpb,distance,speed,tt,split,travel time,d1,tt_next_intersection,d1_next_intersection,Prev_tt_id,nextIntersection
0,9005,1,2,30503,-1,0,5,1850,3,1.541667,0,51.45,0.0,60,0.0,190,12.9,385,0,1
1,9005,1,6,30504,1,0,5,1850,3,1.541667,662,51.45,12.9,60,0.0,210,0.0,0,30604,-1
2,9006,2,2,30603,0,1,5,1850,3,1.541667,662,51.45,12.9,60,107.0,385,12.8,585,30503,2
3,9006,2,6,30604,2,1,5,1850,3,1.541667,660,51.45,12.8,60,13.0,405,12.9,210,30704,0
4,9007,3,2,30703,1,2,5,1850,3,1.541667,660,51.45,12.8,70,94.0,585,12.8,765,30603,3
5,9007,3,6,30704,3,2,5,1850,3,1.541667,657,51.45,12.8,50,26.0,605,12.8,405,30804,1
6,9008,4,2,30803,2,3,5,1850,3,1.541667,657,51.45,12.8,44,82.0,765,13.1,970,30703,4
7,9008,4,6,30804,4,3,5,1850,3,1.541667,675,51.45,13.1,44,38.0,785,12.8,605,30904,2
8,9009,5,2,30903,3,4,5,1850,3,1.541667,675,51.45,13.1,70,68.0,970,12.8,1170,30803,5
9,9009,5,6,30904,5,4,5,1850,3,1.541667,660,51.45,12.8,70,52.0,990,13.1,785,31004,3


In [7]:
run_ids = [1,2,3,5,4,6,12, 10, 16, 8, 15, 14]
number_of_iteration = 10

aog = np.zeros(( len(run_ids), 2, number_of_iteration))
ns = np.zeros_like(aog)
bw = np.zeros_like(aog)
prog = np.zeros_like(aog)


In [8]:
signal_sql_test = "SELECT [RunID], [Iteration], [SimTime], [SC], [SG], [state] FROM [sim_Andalib].[dbo].[signalchanges] where [StudyID]="+str(sim_id)
signal_df = pd.read_sql(signal_sql_test, cnxn)

if(len(signal_df)==0): print("ERROR")

# 1- green start, 2- yellow start, 0- Red start
signal_df = signal_df.loc[signal_df.state.isin([0, 1])]
signal_df["eog"] = signal_df.SimTime - signal_df.SimTime*signal_df.state
signal_df["bog"] = (signal_df.SimTime - signal_df.SimTime*(1-signal_df.state))

signal_df["eog"] = signal_df["eog"].astype(np.int64)
signal_df["bog"] = signal_df["bog"].astype(np.int64)

In [9]:
delay_sql = "SELECT [RunID],[Iteration],[Time],[No_],[Veh],[Trav_],[Delay_] FROM [sim_Andalib].[dbo].[traveltimes] where [StudyID]="+str(sim_id)
delay_df = pd.read_sql(delay_sql, cnxn)
delay_df.columns = ['RunID','Iteration','time','tt_id', 'vehNo', 'travel_time', 'travel_delay']

In [10]:
def aog_prog(signal, eog_matrix, eog_cycle_num, split_matrix, split_cycle_num, print_output= False):
        green_time = np.zeros((number_of_intersection,2, (number_of_cycle+1)*cycle_length))
        
        for link_ in range(len(input_parameters)):

                intersection_num = input_parameters["Intersection Number"][link_]
                intersection_ = input_parameters["intersection_id"][link_]

                sg_ = input_parameters.sg_id[link_]

                signal_ = signal.loc[(signal.SC==intersection_num) & (signal.SG==sg_)]

                if(sg_==2): bound = 0
                else: bound  = 1

                for step in range(0, len(signal_), 2):
                        bog = signal_.iloc[step, 5]

                        if(step<len(signal_) - 1):
                                eog = signal_.iloc[step+1, 4]

                                if(eog>0.1): 
                                        eog_matrix[intersection_ - 1][bound][eog_cycle_num[intersection_ - 1][bound]] = eog%cycle_length
                                        eog_cycle_num[intersection_ - 1][bound] += 1

                        else: eog = simulation_time

                        if(bog>0.1 and eog!=simulation_time):
                                split_matrix[intersection_ - 1][bound][split_cycle_num[intersection_ - 1][bound]] = eog - bog
                                split_cycle_num[intersection_ - 1][bound] += 1    
                        if(print_output): print(bog, " ", eog)
                        green_time[intersection_ -1,bound, bog:eog] = 1

        green_distribution = green_time.reshape(number_of_intersection,2, (number_of_cycle+1), cycle_length)

        cycle_sum = np.ones((cycle_length, 1)) * number_of_cycle
        cycle_sum[:cycle_remainder, 0] += 1

        return np.sum(green_distribution, axis = 2)/cycle_sum.T
        

In [11]:
def green_bw(split_matrix,split_cycle_num, eog_matrix, eog_cycle_num, final_green_time):
        expcted_split = np.round(np.sum(split_matrix, axis = 2)/ split_cycle_num, 0)
        expcted_split = np.minimum(expcted_split, cycle_length)

        for i in range(number_of_intersection):
                for j in range(2):
                        split_current = split_matrix[i, j, :split_cycle_num[i][j]]
                        expcted_split2 = int(np.round(stats.mode(split_current)[0], 0))

                        eog = int(stats.mode(eog_matrix[i,j, :eog_cycle_num[i][j]])[0])
                        bog = (eog - int(expcted_split[i][j]))%cycle_length

                        if(eog>bog): 
                                final_green_time[i, j, bog:eog] = 1
                        else: 
                                final_green_time[i, j, bog:cycle_length] = 1
                                final_green_time[i, j, 0:eog] = 1


In [12]:
def shift_green(final_green_time):
        link_ = 0
        for i in range(number_of_intersection):
                for j in range(2):
                        tt = int(input_parameters["travel time"][link_])
                        placeHolder_green  = np.hstack((final_green_time[i, j, cycle_length - tt: cycle_length], \
                                                        final_green_time[i, j, 0: cycle_length - tt]))

                        final_green_time[i, j, :] = placeHolder_green[:]

                        link_ += 1

In [13]:
### capacity_matrix
def capacity_calculate(capacity, green_aog):

        for link_ in range(len(input_parameters)):
                intersection_ = input_parameters["intersection_id"][link_]
                sg_ = input_parameters.sg_id[link_]
                vpb = input_parameters.vpb[link_]

                if(sg_==2): bound = 0
                else: bound  = 1
                capacity[intersection_ -1,bound, :] = green_aog[intersection_ -1,bound, :] * vpb * number_of_cycle

In [14]:
def vehicle_arrival_calculate(travel_time):
        vehicle_arrival = np.zeros((number_of_intersection,2, (number_of_cycle+1)*cycle_length))
        for link_ in range(len(input_parameters)):
                if(input_parameters.loc[link_, "sg_id"]==2): bound = 0
                else: bound  = 1

                intersection_ = input_parameters.loc[link_, "intersection_id"] -1

                if(input_parameters.loc[link_, "LinkFromInt"]==-1): 
                        vehicle_arrival[intersection_,bound, :] = 0
                        continue


                travel_time_intersection = input_parameters.loc[link_, "travel_time"]
                travel_time_id = input_parameters.loc[link_, "tt_id"]

                veh_tt = travel_time.loc[travel_time.tt_id == travel_time_id].copy()    
                veh_tt.loc[:, "timestamp"] = np.round((veh_tt.time - veh_tt['travel_time'] + travel_time_intersection), 0)
                veh_tt.timestamp = veh_tt["timestamp"].astype(np.int64)

                unique, counts = np.unique(veh_tt.timestamp, return_counts=True)
                vehicle_arrival[intersection_,bound, unique] = counts

        vehicle_arrival_reshape = vehicle_arrival.reshape(number_of_intersection,2, (number_of_cycle+1), cycle_length)
        return np.sum(vehicle_arrival_reshape, axis = 2)


In [16]:
def arrival_profile_optimize(bin_GAdj, bin_VAdj): 

        bin_AOG = np.multiply(bin_GAdj, bin_VAdj) 
        sys_AOG = np.sum(bin_AOG, axis = (0, 2))

        return sys_AOG

In [17]:
### badnwidth

def maximize_bandwidth(bin_GAdj):
        return np.sum(np.prod(bin_GAdj, axis = 0), axis = 1)

In [18]:
def number_of_stops(capacity_matrix, bin_VAdj):
    ### If (overall) volume exceeds capacity this method will not work
    
        queue= np.zeros(bin_VAdj.shape)

        veh_left = bin_VAdj - capacity_matrix

        ### first iteration
        for i in range(cycle_length):
                queue[:, :, i] = np.maximum(queue[:, :, i-1] + veh_left[:, :, i], 0)
    
    ### second iteration
        total_stops = np.zeros(2)
    
        for i in range(cycle_length):
                queue[:, :, i]   = np.maximum(queue[:, :, i-1] + veh_left[:, :, i], 0)

                stops = np.maximum(bin_VAdj[:, :, i] - np.maximum(capacity_matrix[:, :, i] - queue[:, :, i-1], 0), 0)
                total_stops += np.sum(stops, axis = 0)

        return total_stops

In [19]:
def plot_distributions(probability_of_green, vehicle_distribution):
        signal_seq = np.arange(0,cycle_length, 1)
        veh_seq = np.arange(0,cycle_length, 1)

        cols = (number_of_intersection)
        rows = 2

        fig, axes = plt.subplots(nrows = rows, ncols = cols,sharey=True, sharex = True, figsize=(32, 4))

        fig.tight_layout()
    
        for link_ in range(len(input_parameters)):
        
                i = input_parameters["intersection_id"][link_]-1

                if(input_parameters.loc[link_, "sg_id"]==2): j = 1
                else: j  = 0

                ax = axes[j][i]
                ax.set_title("Intersection Number: " + str(i+1) + " SG: " +str(input_parameters.loc[link_, "sg_id"]))
                ax.grid(which = 'both', linestyle = 'dashed')
                ax.grid(b=True, which='minor', alpha=0.2)
                ax.minorticks_on()
                ax.set_axisbelow(True)

                ax.bar(signal_seq,probability_of_green[i][1-j], width = 1, color = 'green', alpha = 0.8)

                ax2=ax.twinx()
                ax2.bar(veh_seq,vehicle_distribution[i][1-j], width = 1, color = 'black', alpha = 0.6)

In [20]:
for rid  in range(len(run_ids)): 
        run_id = run_ids[rid]
        for iter_number in range(number_of_iteration):
                signal = signal_df.loc[(signal_df.RunID==run_id) & (signal_df.Iteration==iter_number+1)  & (signal_df.SimTime<=3600), ['SimTime', 'SC', 'SG', 'state', 'eog', 'bog']].copy()
                travel_time = delay_df.loc[(delay_df.RunID==run_id) & (delay_df.Iteration==iter_number+1) & (delay_df.time>=simulation_start) &  (delay_df.time<=3600)].copy() #
                
                signal.sort_values(by = "SimTime", inplace = True)
                travel_time.sort_values(by = "time", inplace = True)
                
                eog_matrix = np.zeros((number_of_intersection, 2, number_of_cycle+10))
                split_matrix = np.zeros((number_of_intersection, 2, number_of_cycle+10))

                eog_cycle_num = np.zeros((number_of_intersection, 2), dtype = np.int16)
                split_cycle_num = np.zeros((number_of_intersection, 2), dtype = np.int16)
                
                ### initial probability of green matrix
                print_output = 0
                prog_band = aog_prog(signal, eog_matrix, eog_cycle_num, split_matrix, split_cycle_num, print_output)
                
                ### green splits for bandwidth calculation
                green_band = np.zeros((number_of_intersection,2, cycle_length))
                green_bw(split_matrix,split_cycle_num, eog_matrix, eog_cycle_num, green_band)
                
                shift_green(green_band)
                
                ### shifted PrOG to calculate probability of meeting at green

                shifted_prog = np.array(prog_band)
                shift_green(shifted_prog)
                
                ### capacity matrix
                capacity = np.zeros((number_of_intersection,2, cycle_length))
                capacity_calculate(capacity, prog_band)
                
                ### vehicle arrival
                vehicle_arrival = vehicle_arrival_calculate(travel_time)

                ### Results
                aog[rid, :, iter_number] = arrival_profile_optimize(prog_band, vehicle_arrival)
                ns[rid, :, iter_number] = number_of_stops(capacity, vehicle_arrival)
                bw[rid, :, iter_number] = maximize_bandwidth(green_band)
                prog[rid, :, iter_number] = maximize_bandwidth(shifted_prog)
                

In [25]:
aog_mean = np.round(np.mean(aog, axis = 2), 2)
ns_mean = np.round(np.mean(ns, axis = 2), 2)
bw_mean = np.round(np.mean(bw, axis = 2), 2)
prog_mean = np.round(np.mean(prog, axis = 2), 2)

In [28]:
result_array = np.hstack([aog_mean,ns_mean, bw_mean,  prog_mean])
result_df = pd.DataFrame(result_array, columns = ["aog_eb", "aog_wb", "Stop_eb", "Stop_wb", "bw_eb", "bw_wb", "prog_eb", "prog_wb"])
result_df.loc[:, "Scenario"] = ["Zero-offset", "AOG" , "Modified-AOG", "AOG-CV", "Delay" , "Number of Stops" , "Delay + Stops", "PrOG", "Kell Method", "BW" , "Maxband", "Multiband" ]

result_df.loc[:, "AOG"] = result_df["aog_eb"] + result_df["aog_wb"]
result_df.loc[:, "Stop"] = result_df["Stop_eb"] + result_df["Stop_wb"]
result_df.loc[:, "BW"] = result_df["bw_eb"] + result_df["bw_wb"]
result_df.loc[:, "prog"] = result_df["prog_eb"] + result_df["prog_wb"]

# result_df = result_df.reindex(columns = ["Zero-offset", "AOG" , "Modified-AOG", "AOG-CV", "Delay" , "Number of Stops" , "Delay + Stops", "PrOG", "Kell Method", "BW" , "Maxband", "Multiband" ])

In [29]:
result_df

,aog_eb,aog_wb,Stop_eb,Stop_wb,bw_eb,bw_wb,prog_eb,prog_wb,Scenario,AOG,Stop,BW,prog
0,7345.00,6670.24,3052.98,2136.71,0.0,0.0,1.04,0.18,Zero-offset,14015.24,5189.69,0.0,1.22
1,7863.71,6374.84,2413.32,2596.20,16.9,0.0,13.17,1.64,AOG,14238.55,5009.52,16.9,14.81
2,7961.05,6373.00,2304.66,2581.45,20.9,0.0,16.21,0.83,Modified-AOG,14334.05,4886.11,20.9,17.04
3,7877.28,6346.50,2447.82,2643.08,18.0,0.0,13.94,0.71,AOG-CV,14223.78,5090.90,18.0,14.65
4,7704.51,6561.36,2560.59,2368.17,8.3,2.5,6.54,6.87,Delay,14265.87,4928.76,10.8,13.41
5,7775.51,6569.68,2415.61,2321.19,9.3,2.6,7.13,6.74,Number of Stops,14345.19,4736.80,11.9,13.87
6,7760.70,6556.99,2484.96,2339.35,10.3,0.8,8.05,5.95,Delay + Stops,14317.69,4824.31,11.1,14.00
7,6979.57,7452.69,3842.50,1674.86,2.0,51.9,3.88,43.55,PrOG,14432.26,5517.36,53.9,47.43
8,7250.11,6954.30,3506.86,2153.30,12.8,15.1,10.98,12.83,Kell Method,14204.41,5660.16,27.9,23.81
9,8394.82,6416.29,1823.67,3144.27,39.9,19.9,28.62,13.78,BW,14811.11,4967.94,59.8,42.40
